## The following code uses the Pythonspaudiopy package
- Docs: https://spaudiopy.readthedocs.io/en/latest/index.html
- GitHub: https://github.com/chris-hld/spaudiopy

## Tools
We use three parameters to locate an HRTF:
1. *Azimuth*: angle between position and sound location on the $xy$-plane
2. *Elevation*: angle between position and sound location on the $xz$-plane
3. *Time* or *Frequency*: time period or frequency of emitted sound w.r.t. actual position

Math tools:
- *Haversine distance*: Consider two points $x_1$ and $x_2$ on a sphere with respective latitudes and longitudes $(\varphi_1,\varphi_2)$ and $(\theta_1,\theta_2)$. The Haversine distance $D(x_1,x_2)$ is the angular distance between them on the surface of the sphere given by $$D(x_1,x_2)=2\arcsin\sqrt{\sin^2\left(\frac{\varphi_2-\varphi_1}{2}\right)+\cos x_1\cos y_1\sin^2\left(\frac{\theta_2-\theta_1}{2}\right)}.$$ We use this distance when wanting to find the closest HRIR point from a grid.

## Imports

In [ ]:
# Check spaudiopy.sig functions to open MONO signal and play with HRIRs
import spaudiopy as spa
from spaudiopy.sig import MonoSignal as ms
from spaudiopy.sig import MultiSignal as stereo 
import spaudiopy.process as sproc

import scipy.signal
from scipy.io import wavfile
from IPython.display import Audio

fs=44100 # sampling rate
import numpy as np

## Test delay on stereo file

In [ ]:
# Initial test
stereo_sample = stereo.from_file("ocean_eyes.wav")
s1, s2 = stereo_sample.get_signals()

In [ ]:
delay = np.zeros(len(stereo_sample), dtype=int)
delay[int(fs*0.03)] = 1
delayed_s2 = scipy.signal.convolve(s2, delay)[:len(stereo_sample)]
delayed_stereo = stereo([s1, delayed_s2], fs=fs)

In [ ]:
display(Audio(stereo_sample, rate=fs)) # original sample
#display(Audio(delayed_stereo, rate=fs))

In [ ]:
def get_stereo_tracks(music_file):
    return stereo.from_file(music_file).get_signals()

def delay_signal(s, delay):
    _len_sample = len(s)
    d_arr = np.zeros(_len_sample, dtype=int)
    d_arr[int(fs*delay)] = 1
    return scipy.signal.convolve(s, d_arr)[:_len_sample]

def spatialize_signal(s, delay):
    _len_sample = len(s)
    d_arr = np.zeros(_len_sample, dtype=int)
    d_arr[0] = 1
    d_arr[int(fs*delay)] = 1
    return scipy.signal.convolve(s, d_arr)[:_len_sample]


In [ ]:
MS_DELAY = 25 # delay in [ms] to create surrounding effect

def spatialise_sound_side(music_file, left=False):
    """Create a spatialised version of the given input file
        @param music_file: stereo music file
        @param left: creates the delay to the left or the right given this value
    """
    s1,s2 = get_stereo_tracks(music_file)
    if left:
        delayed_s1 = delay_signal(s1, MS_DELAY / 1000)
        return stereo([delayed_s1, s2], fs=fs)
    delayed_s2 = delay_signal(s2, MS_DELAY / 1000)
    return stereo([s1, delayed_s2], fs=fs)

In [ ]:
s_left = spatialise_sound("ocean_eyes.wav")
s_left.save("ocean_eyes_spatialised.wav")
display(Audio(s_left, rate=fs))
#s_right = spatialise_sound("ocean_eyes.wav", True)
#display(Audio(s_right, rate=fs))

## Reverb for increased spatial effect
Create simultaneous small delay effects and combine them together in a new signal.

In [ ]:
# TODO: WORK IN PROGRESS
def reverb(music_file):
    s1,s2 = get_stereo_tracks(music_file)
    tracks = [s1]
    for _ in range(5):
        delayed_s2 = delay_signal(s2, 5 / 1000)
        tracks.append(delayed_s2)
        s2 = delayed_s2
    return stereo(tracks, fs=fs)

s_rev = reverb("ocean_eyes.wav")
display(Audio(s_rev, rate=fs))

In [ ]:
# TODO: WORK IN PROGRESS
def echo(s):
    _filter = [1]
    for i in range(1, len(s)):
        _filter.append(_filter[-1]*0.5)
    return _filter

def successive_echo(s):
    _filter = [1]
    for i in range(1, len(s)):
        if i % fs/2 == 0:
            _filter.append(1)
        else:
            _filter.append(_filter[-1]*0.99)
    return _filter

def successive_delays(s, delay):
    return scipy.signal.convolve(s, echo(s))[:len(s)]

def reverb_v2(music_file):
    s1,s2 = get_stereo_tracks(music_file)
    delayed_s1 = delay_signal(s1, MS_DELAY / 1000)
    delayed_s2 = successive_delays(s2, MS_DELAY / 1000)
    return stereo([s1, s2, delayed_s2], fs=fs)

s_rev2 = reverb_v2("ocean_eyes.wav")
display(Audio(s_rev2, rate=fs))

## Frequency plots
We wish to visualise the main differences between the two frequency plots of both the original stereo file and the generated spatialised one.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
def plot_freqs(wav_file, start, end):
    fs, data = wavfile.read(wav_file)
    data = data[:,0]
    plt.figure()
    data_to_plot = data[fs*start:fs*end]
    plt.plot(data_to_plot)
    plt.xlabel('Sample Index')
    plt.ylabel('Amplitude')
    plt.title('Waveform of ' + wav_file)
    plt.show()
    return data_to_plot

In [ ]:
f_ster = plot_freqs('ocean_eyes.wav', 30, 31)
f_spat = plot_freqs('ocean_eyes_spatialised.wav', 30, 31)

## Use of localisation cues
Add direction to sound source

In [ ]:
avg_ear_distance = 0.21 # 21cm
speed_of_sound = 343 # in m/s

def fusion(s1, s2):
    assert(len(s1) == len(s2))
    return [s1[i]/2 + s2[i]/2 for i in range(len(s1))]

def compute_delay_from_angle(rad):
    return avg_ear_distance * np.sin(rad) / speed_of_sound

def attenuate(sig, direction):
    return [e*np.cos(direction/4) for e in sig]

def localise_sound(stereo_music_file, direction, left=True):
    """Change location of the sound source
        @param stereo_music_file: music file we wish to use
        @param direction: where we want the sound to come from
        @param left: False if we want the sound to come from the right, True o/wise
    """
    s1,s2 = get_stereo_tracks(stereo_music_file)
    sample_song_mono = fusion(s1, s2)
    
    maxmono = max(sample_song_mono)
    _len_sample = len(sample_song_mono)
    
    mono_norm = [sample_song_mono[i]/(maxmono+1) for i in range(_len_sample)]
    delayed = delay_signal(mono_norm, compute_delay_from_angle(direction))
    
    if left:
        return stereo([mono_norm, attenuate(delayed, direction)], fs=fs) # sound to the left
    return stereo([attenuate(delayed, direction), mono_norm], fs=fs) # sound to the right

In [ ]:
s_west = localise_sound("ocean_eyes.wav", np.pi/2)
display(Audio(s_west, rate=fs))

In [ ]:
s_east = localise_sound("ocean_eyes.wav", np.pi/2, False)
display(Audio(s_east, rate=fs))

In [ ]:
# WORK IN PROGRESS
def localise_spacialise_sound(music_file, direction, left=True):
    s1,s2 = get_stereo_tracks(music_file)
    sample_song_mono = fusion(s1, s2)
    
    maxmono = max(sample_song_mono)
    _len_sample = len(sample_song_mono)
    
    mono_norm = [sample_song_mono[i]/(maxmono+1) for i in range(_len_sample)]
    angle = compute_delay_from_angle(direction)
    delayed = delay_signal(attenuate(mono_norm, angle), angle)
        
    mono_norm_spazialized = spatialize_signal(mono_norm, MS_DELAY / 1000)
    delayed_spazialized = spatialize_signal(delayed, MS_DELAY / 1000) 
    
    if left: 
        return stereo([mono_norm_spazialized, delayed_spazialized], fs=fs)
    return stereo([delayed_spazialized, mono_norm_spazialized], fs=fs)

In [ ]:
sound = localise_spacialise_sound("ocean_eyes.wav", np.pi/2, False)
display(Audio(sound, rate=fs))

In [ ]:
sound = localise_spacialise_sound("ocean_eyes.wav", np.pi/2, True)
display(Audio(sound, rate=fs))